Model 1 : FASTtext
https://github.com/facebookresearch/fastText
               
 A. Joulin, E. Grave, P. Bojanowski, T. Mikolov, Bag of Tricks for Efficient Text Classification  https://arxiv.org/abs/1607.01759

In [1]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip && unzip v0.9.2.zip && pip install fastText-0.9.2/ 
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rajDxvxqZkth8BW6j-1OvQ585W546sw9'
import pandas as pd
df = pd.read_excel (r'/content/uc?export=download&id=1rajDxvxqZkth8BW6j-1OvQ585W546sw9')
df2='__label__'+df['target'].astype(str) +" " +df['review_text'].astype(str)
tfile = open('test.txt', 'a')
tfile.write(df2.to_string(index = False))
tfile.close()
!wc test.txt 
!cat test.txt | sed -e "s/[[:upper:]]*/\L&/g ; s/\+[^а-я]/ /g;s/ \+/ /"> test0.txt
!head -n 64227 test0.txt > test.train
!tail -n 13056 test0.txt > test.valid
import fasttext
model=fasttext.train_supervised(input="test.train", epoch=25, wordNgrams=5)
model.save_model("kino.bin")
model.test("test.valid")

--2020-09-11 08:05:45--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [following]
--2020-09-11 08:05:45--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip’

v0.9.2.zip              [  <=>               ]   4.17M  13.5MB/s    in 0.3s    

2020-09-11 08:05:45 (13.5 MB/s) - ‘v0.9.2.zip’ saved [4369852]

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
   creating: fastText-0.9.2/
   creating: fastText-0.9.2/.circleci/
  inflating: fastText

(13056, 0.7228094362745098, 0.7228094362745098)

In [2]:
model.test("test.valid" ,k=3)

(13056, 0.3333333333333333, 1.0)

In [8]:
model.test("test.valid" )

(13056, 0.7228094362745098, 0.7228094362745098)

Model 2: CNN (уже использовала  GPU as a backend for free for 12 hours ) 

In [ ]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='theano' # Why theano why not
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline
def clean_str(string):
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    
    return string.strip().lower()
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)
print('No. of unique classes',len(set(df['target'])))
macronum=sorted(set(df['target']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

df['target']=df['target'].apply(fun)
texts = []
labels = []


for idx in range(df.review_text.shape[0]): 
  text = BeautifulSoup(str(df.review_text[idx])) 
  texts.append(clean_str(str(text.get_text().encode())))

for idx in df['target']:
    labels.append(idx)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Number of Unique Tokens',len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of Data Tensor:', data.shape)
print('Shape of Label Tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]



Shape of dataset  (60198, 6)
Index(['target', 'movie_link', 'author_id', 'author_name', 'review_subtitle',
       'review_text'],
      dtype='object')
No. of unique classes 3
Number of Unique Tokens 1311178
Shape of Data Tensor: (60198, 1000)
Shape of Label Tensor: (60198, 3)


In [ ]:
!wget http://vectors.nlpl.eu/repository/20/195.zip
!unzip 195.zip
embeddings_index = {}
f = open('vocab.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word) 
 
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,trainable=True)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(len(macronum), activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Simplified convolutional neural network")
model.summary()
cp=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

--2020-09-01 04:21:10--  http://vectors.nlpl.eu/repository/20/195.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206977027 (197M) [application/zip]
Saving to: ‘195.zip’

195.zip             100%[===================>] 197.39M  70.9MB/s    in 2.8s    

2020-09-01 04:21:13 (70.9 MB/s) - ‘195.zip’ saved [206977027/206977027]

Archive:  195.zip
  inflating: meta.json               
  inflating: model.hdf5              
  inflating: options.json            
  inflating: README                  
  inflating: vocab.txt               
Total 10000 word vectors in Glove 6B 100d.
Simplified convolutional neural network
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000)]            0         
________

In [ ]:
history=model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=2, batch_size=2,callbacks=[cp])

Epoch 1/2
24080/24080 [==============================] - ETA: 0s - loss: 0.7200 - acc: 0.7706
Epoch 00001: val_acc improved from -inf to 0.77747, saving model to model_cnn.hdf5
24080/24080 [==============================] - 4617s 192ms/step - loss: 0.7200 - acc: 0.7706 - val_loss: 0.7333 - val_acc: 0.7775
Epoch 2/2
24080/24080 [==============================] - ETA: 0s - loss: 0.8071 - acc: 0.7689
Epoch 00002: val_acc improved from 0.77747 to 0.77781, saving model to model_cnn.hdf5
24080/24080 [==============================] - 3955s 164ms/step - loss: 0.8071 - acc: 0.7689 - val_loss: 0.7116 - val_acc: 0.7778
